In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from matplotlib.colors import LogNorm
import tqdm
import pickle 
from numba import jit



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
files = [
    "./ntupleStubsBmtfGmt_LS133.root",
    "./ntupleStubsBmtfGmt_LS263.root",
    "./ntupleStubsBmtfGmt_LS264.root",
    "./ntupleStubsBmtfGmt_LS306.root",
]

stub_bmtf_gmt_features = [
    'orbit', 'bx', 'nStubs', 'stubHwPhi', 'stubHwPhiB', 'stubHwQual', 'stubHwEta', 'stubHwQEta', 'stubStation', 'stubWheel', 'stubSector', 'stubTag',
    'nBmtfMuons', 'bmtfHwPt', 'bmtfHwPtu', 'bmtfHwPhi', 'bmtfHwEta', 'bmtfPt', 'bmtfPtu', 'bmtfPhi', 'bmtfEta', 'bmtfHwSign', 'bmtfHwSignValid', 
    'bmtfHwDXY', 'bmtfHwQual', 'bmtfProcessor', 
    'nGmtMuons', 'gmtHwPt', 'gmtHwPtu', 'gmtHwPhi', 'gmtHwEta', 'gmtHwPhiAtVtx', 'gmtHwEtaAtVtx', 'gmtPt', 'gmtPtu', 'gmtPhi', 'gmtEta', 
    'gmtPhiAtVtx', 'gmtEtaAtVtx', 'gmtCharge', 'gmtDxy', 'gmtQual', 'gmtTfIndex']

stub_bmtf_gmt_list = []
for file in files:
    with uproot.open(file) as f:
        evs = f["bmtfNtuplizer/Events"]
        temp_awk = evs.arrays(stub_bmtf_gmt_features, library="ak")
        stub_bmtf_gmt_list.append(temp_awk)
        
bmtf_gmt = ak.concatenate(stub_bmtf_gmt_list, axis=0)



In [3]:
nStubs = bmtf_gmt.nStubs
stubsStation = bmtf_gmt.stubStation
stubsSector = bmtf_gmt.stubSector
stubsWheel = bmtf_gmt.stubWheel
stubsPhiB = bmtf_gmt.stubHwPhiB
stubsHwQual = bmtf_gmt.stubHwQual

In [36]:
#123, 258, 287, 
'''for i in range(1000):
   print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i], stubsHwQual[i])'''

'''i, j = 123,124
print(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])
print(j, nStubs[j], stubsStation[j], stubsSector[j], stubsWheel[j])'''

'i, j = 123,124\nprint(i, nStubs[i], stubsStation[i], stubsSector[i], stubsWheel[i])\nprint(j, nStubs[j], stubsStation[j], stubsSector[j], stubsWheel[j])'

In [25]:
Sta = np.array([1])
StaP1 = np.array([1])

def CheckStation(Station, StationP1):

   Diff = Station - StationP1

   return Diff ==  -1

Bool = CheckStation(Sta, StaP1)
Bool

array([False])

In [5]:
Features = [nStubs, stubsStation, stubsSector, stubsWheel, stubsPhiB, stubsHwQual]

In [43]:
#Controlla che la stub successiva si trovi nella stazione dopo
def CheckStation(Station, StationP1):

   Diff = Station - StationP1
   Index = np.where(Diff == -1)[0]

   return Index

#Controlla che la Wheel successiva sia +- 1 della wheel corrente, tranne per le wheel estremali
def CheckWheel(Wheel, WheelP1):
   
   IndexList = []

   if (Wheel == -2):
      Index = list(np.where((WheelP1 == -1) | (WheelP1 == -2))[0])
      IndexList.extend(Index)

   elif (Wheel == 2):
      Index = list(np.where((WheelP1 == 1) | (WheelP1 == 2))[0])
      IndexList.extend(Index)

   else:
      Diff = Wheel - WheelP1
      Index = list(np.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return np.array(IndexList)

#Controlla che il settore successivo sia adiacente al settore corrente. Applica operazione ciclica per settori 11 e 0
def CheckSector(Sector, SectorP1):
   
   IndexList = []

   if (Sector == 11):
      Index = list(np.where((SectorP1 == 10) | (SectorP1 == 11) | (SectorP1 == 0))[0])
      IndexList.extend(Index)

   elif (Sector == 0):
      Index = list(np.where((SectorP1 == 11) | (SectorP1 == 0)| (SectorP1 == 1))[0])
      IndexList.extend(Index)

   else:
      Diff = Sector - SectorP1
      Index = list(np.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return np.array(IndexList)

In [72]:
Sta = np.array([1, 1])
StaP1 = np.array([3])
Sec = np.array([1, 3])
SecP1 = np.array([3])
Whe = np.array([0, 0])
WheP1 = np.array([1])

ChkStation = CheckStation(Sta[0], StaP1)
ChkSector = CheckSector(Sec[0], SecP1)
ChkWheel = CheckWheel(Whe[0], WheP1)

print(ChkStation, ChkSector, ChkWheel)

[] [] [0]


In [68]:
def SearchHSCP(Features, Number):
   
   HSCPs = []
   GlobalMatch = []
   GlobalStation, GlobalStationP1 = [], []
   GlobalSector, GlobalSectorP1 = [], []
   GlobalWheel, GlobalWheelP1 = [], []

   GlobalChkStation, GlobalCHkSector, GlobalChkWheel = [], [], []


   for i in tqdm.tqdm(range(Number - 1)):

      #Feature: 0 == Stubs, 1 == Station, 2 == Sector, 3 == Wheel, 4 == PhiB, 5 == Quality
      

      #TODO controllo duplicati
      if (Features[0][i] == Features[0][i+1]):

         LocalMatch = []

         LocalStation, LocalStationP1 = [], []
         LocalSector, LocalSectorP1 = [], []
         LocalWheel, LocalWheelP1 = [], []


         for j in range(Features[0][i]):

            ChkStation = CheckStation(Features[1][i][j], Features[1][i+1])
            ChkSector = CheckSector(Features[2][i][j], Features[2][i+1])
            ChkWheel = CheckWheel(Features[3][i][j], Features[3][i+1])

            if (len(ChkStation) >= 1 and len(ChkSector) >= 1 and len(ChkWheel) >= 1):
               LocalMatch.append(True)

            elif (len(ChkStation) == 0 or len(ChkSector) == 0 or len(ChkWheel) == 0):
               LocalMatch.append(False)

            else:
               LocalMatch.append(False)

         
            GlobalMatch.append(LocalMatch)

      else:
         continue

   
   return GlobalMatch

In [69]:
test = SearchHSCP(Features, 100)

100%|██████████| 999/999 [00:01<00:00, 541.47it/s]


In [70]:
test

[[False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [True],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False, False],
 [False, False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False